## Welcome to the CoQuí Many-Body Electronic Structure Tutorial

CoQuí is a C++/MPI scientific package for **ab initio many-body electronic structure** calculations **beyond** density functional theory (**DFT**). On top of the C++ implementations, CoQuí provides thin **Python wrappers** that enable direct exchange of input and output data, making it straightforward to couple with external packages. 

> 💡 There is another way for users to interact with CoQuí by writing input files in the toml format. [Examples](https://github.com/AbInitioQHub/coqui/tree/main/examples) in this mode are available. 

For the interest of time, in this *condensed* tutorial series we will not attempt to cover all features of CoQuí or discuss APIs and input parameters in detail. Instead, the focus will be on the **concept of CoQuí’s code structure**, illustrating its capabilities and simplicity. More comprehensive [examples](https://github.com/AbInitioQHub/coqui/tree/main/examples) are available in [CoQuí official Github organization](https://github.com/AbInitioQHub). 

Here, the current introduction notebook will: 
- Ensure you can run CoQuí in the provided environment via the Python interface
- Introduce the basic execution workflow (MPI + Python)
- Outline the general workflow of many-body simulations in CoQuí
- Outline how this tutorial series is structured

## 🐸 CoQuí "Hello World"

Let’s start by importing **CoQuí** into Python for the very first time.  

👉 Run the next cell to:
- Import the `coqui` package,  
- Print the installed **CoQuí version**,  
- Show the **path** where the Python bindings are loaded from,  
- Display the current **Git branch** and **commit hash** that your build corresponds to.

In [1]:
import os, coqui

# CoQui version 
print(f"CoQui version: {coqui.__version__}")
# CoQui git information
print(f"Path for the CoQui Python library: {os.path.dirname(coqui.__file__)}")
print(f"GIT branch: {coqui.COQUI_GIT_BRANCH}")
print(f"GIT hash: {coqui.COQUI_GIT_HASH}\n")

CoQui version: 0.1.0
Path for the CoQui Python library: /mnt/home/cyeh/Projects/coqui/build/main/llvm19/install/lib/python3.11/site-packages/coqui
GIT branch: main
GIT hash: 814a23d



### 🔹 MPI Environment in CoQuí
The CoQuí C++ library requires that an MPI environment be initialized **before** any CoQui routine is called. 
When using CoQuí from Python, this MPI environemnt must already be active — for example, via `mpi4py` or another MPI-aware Python framework - before invoking any CoQui routines. 

Once the MPI environment is initialized, CoQuí routines become aware of it through a `coqui.MpiHandler` object. This class encapsulates information about the initialized MPI context and **must be passed, direclty or indirectly, to any CoQui routine** to ensure that MPI communication is handled consistently across the entire simulation workflow, from Hamiltonian generation to many-body simulations, to post-processing.  

A minimum example: 
```python
from mpi4py import MPI    # The MPI environment is initialized here
import coqui

# 1) Create a CoQuí's MPI handler
coqui_mpi = coqui.MpiHandler()

# 2) (Optional) Set global logging verbosity for CoQui
coqui.set_verbosity(coqui_mpi, output_level=1)

# Print MPI state from the root process
if coqui_mpi.root():
    print(coqui_mpi)
```
#### ❓**What happens?**
1. **MPI environment is initialized first**:  
   `mpi4py` sets up the global MPI context before CoQuí functions are called.
3. **Create `coqui.MpiHandler` once and reused it**
   - Captures the current MPI environment and provides access to:
     - Global communicator (all ranks)
     - Intranode communicator (within a single node)
     - Internode communicator (across nodes)
   - Must be constructed **first** and then passed to all CoQuí routines that involve MPI parallelization, directly or indirectly. 

> ⚠️**Notes**: Even in the serial mode, `MpiHandler` is still required to keep the interface consistent.

### 🔹 Running a CoQuí Python script in parallel

To run a CoQuí Python script in parallel, use:
```bash
mpirun -np X python {YOUR_COQUI_PYTHON_SCRIPT}.py
```
This will launch CoQuí with `X` processes. 

### ▶️ Hands-on 1: CoQuí MPI configuration

In this first hands-on, we will check that CoQuí runs correctly with MPI both in the notebooks and on terminal. 

1. Run the following Python snapshot in the notebook. How many processors do you have? 
```python
from mpi4py import MPI    # The MPI environment is initialized here
import coqui

# 1) Create a CoQuí's MPI handler
coqui_mpi = coqui.MpiHandler()

# 2) (Optional) Set global logging verbosity for CoQui
coqui.set_verbosity(coqui_mpi, output_level=1)

# Print MPI state from the root process
if coqui_mpi.root():
    print(coqui_mpi)
```
2. Next, open a terminal in the tutorial environemnt
2. Save the above Python snapshot to `coqui_hello_world.py` 
3. Run the script in parallel with the number of processors you have. 

If everything is working correctly, you should see CoQuí reports the same  MPI configuration. 

In [2]:
from mpi4py import MPI    # The MPI environment is initialized here
import coqui

# 1) Create a CoQuí's MPI handler
coqui_mpi = coqui.MpiHandler()

# 2) (Optional) Set logging verbosity for this run
coqui.set_verbosity(coqui_mpi, output_level=1)

# Print MPI state from the root process
if coqui_mpi.root():
    print(coqui_mpi)

CoQuí MPI state
-----------------
  Global communicator: rank 1 / 1
  Internode communicator: rank 1 / 1
  Intranode communicator: rank 1 / 1


--------------------------------------------------------------------------
Ignoring value for oob_tcp_if_exclude on ccqlin065 (10.250.112.0/20: Did not find interface matching this subnet).
(You can safely ignore this message.)
--------------------------------------------------------------------------


## 🐸 The Many-Body Simulation Workflow in CoQuí

Before we dive into features in CoQuí, it’s helpful to see how the different pieces of CoQuí fit together in a typical many-body electronic structure simulation.

<figure style="text-align: center;">
  <img src="../images/coqui_workflow.png" alt="Workflow of many-body electronic structure simulations in CoQuí" width="70%">
  <figcaption><em>Figure&nbsp;1:</em> General workflow of many-body electronic structure simulations in CoQuí.</figcaption>
</figure>

This workflow can be thought of in three conceptual phases:
1. **Problem Setup**
   - Start from a finished density functional theory (DFT) or other mean-field calculation.
   - Build a mean-field object (`coqui.Mf`) containing **the single-particle information** (structure, k-mesh, bands, orbitals).
   - From `coqui.Mf`, **construct the Coulomb Hamiltonian**, which defines an interacting many-electron problem for the many-body solver.
   - Optionally, generate **maximally localized Wannier functions (MLWFs)** for **embedding or post-processing**.
2. **Simulation**
   - Use the **Coulomb Hamiltonian as input** to the desired electronic structure solver, such as GW.
   - Another feature in this simulation phase is downfolding (e.g., cRPA), which produces effective screened interactions in MLWFs for low-energy modeling.
3. **Post-Processing**
   - Analyze electronic structure results: spectral functions, band interpolation, local densities of states, etc. 

------------

#### How this tutorial series maps to the workflow
In this series, we will cover different parts of the workflow step-by-step:
1. **Intro** (this notebook): Orientation, environment setup, and running CoQuí with MPI.
2. **[Prepare CoQuí inputs from DFT outputs](01_dft_to_coqui_converter.ipynb)**: Converting Quantum ESPRESSO results into CoQuí-readable inputs and generating MLWFs via CoQuí's Wannier90 interface. 
3. **[GW electronic structure](02_gw_electronic_structure.ipynb)**: Constructing the Coulomb Hamiltonian and running GW simulations, plus spectral-function post-processing.
4. **[cRPA screened interactions](03_crpa_screened_interactions.ipynb)**: Using the same Coulomb Hamiltonian for downfolding, generating effective screened interactions in MLWFs. 
5. **[GW+EDMFT embedding (minimum)](04_gw_edmft_minimum.ipynb)**: Combining GW electronic structure and downfolding routines to construct an *ab initio* impurity problem with dynamic interactions in GW+EDMFT. 

In each notebook, we will use a version of the same workflow figure with the current step highlighted, so you can see exactly where we are in the bigger picture. 

> ⚠️ This series of tutorials is designed to be **sequantial**. The order is intentional, as each topic builds upon the previous one. Follow them in the designated order is highly recommended. 